In [1]:
#careful
import pandas as pd #nul
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import json
import geopandas as gpd
from tool_function import *
import altair as alt
from altair import expr, datum
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [112]:
#Sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn import svm
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.metrics import RocCurveDisplay
from sklearn.base import clone
from sklearn import linear_model
import xgboost

# Data Prep

In [3]:
mutations_init = pd.read_pickle('../../data_mutation/geometric_mutations.pkl')

In [4]:
mutations_p = mutation_process(mutations_init)
mutations_p  = niveau_center_connexion(mutations_p)
mutations_p = pop_commune_year(mutations_p)
mutations_p = salary_connexion(mutations_p)

hi!


/Users/muhieddineugo/opt/miniconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [18]:
# Creation smoymai, smoyapt, smoyact and adjustment sbati

original_shape = mutations_p.shape
print(f"Shape before adjustments and filtering: {original_shape}")

mutations = mutations_p.copy()
df_surf = mutations[~((mutations.sbati ==0) & (mutations[['nblocmai', 'nblocapt', 'nblocact']].sum(axis=1) == 0))].copy()

test_set['smoymai'] = df_surf.sbatmai/df_surf.nblocmai
df_surf['smoyapt'] = df_surf.sbatapt/df_surf.nblocapt
df_surf['smoyact'] = df_surf.sbatact/df_surf.nblocact

df_surf = df_surf.reset_index(drop=True)
df_surf.loc[df_surf["sbati"]==0, "sbatmai"] = df_surf.groupby('l_codinsee')['smoymai'].transform(np.nanmedian) * df_surf.nblocmai
df_surf.loc[df_surf["sbati"]==0, "sbatapt"] = df_surf.groupby('l_codinsee')['smoyapt'].transform(np.nanmedian) * df_surf.nblocapt
df_surf.loc[df_surf["sbati"]==0, "sbatact"] = df_surf.groupby('l_codinsee')['smoyact'].transform(np.nanmedian) * df_surf.nblocact
df_surf.loc[df_surf["sbati"]==0, "sbati"] = df_surf[['sbatmai', 'sbatapt', 'sbatact']].sum(axis=1)

df_surf = df_surf.reset_index(drop=True)
df_surf.loc[df_surf["sbati"]==0, 'sbatmai'] = df_surf.groupby('coddep')['smoymai'].transform(np.nanmedian) * df_surf.nblocmai
df_surf.loc[df_surf["sbati"]==0, 'sbatapt'] = df_surf.groupby('coddep')['smoyapt'].transform(np.nanmedian) * df_surf.nblocapt
df_surf.loc[df_surf["sbati"]==0, 'sbatact'] = df_surf.groupby('coddep')['smoyact'].transform(np.nanmedian) * df_surf.nblocact
df_surf.loc[df_surf["sbati"]==0, 'sbati'] = df_surf[['sbatmai', 'sbatapt', 'sbatact']].sum(axis=1)

df_surf['smoymai'] = df_surf.sbatmai/df_surf.nblocmai
df_surf['smoyapt'] = df_surf.sbatapt/df_surf.nblocapt
df_surf['smoyact'] = df_surf.sbatact/df_surf.nblocact

# Filtering out 0 apartments
mutations = df_surf[~(df_surf["nblocapt"]==0)].copy()

#We only want appartements
# wanted_libtypbien = ["UN APPARTEMENT", "APPARTEMENT INDETERMINE", "DEUX APPARTEMENTS", 
                      # "BATI - INDETERMINE : Vefa sans descriptif", "BATI - INDETERMINE : Vente avec volume(s)"]  
# mutations = mutations[mutations.libtypbien.isin(wanted_libtypbien)].copy()
# print(f"\nUnique libtypes: {mutations['libtypbien'].unique()}\n")

# Filtering out small apartments
thresh_sbati = 9
thresh_valeur = 5000
mutations = mutations[(mutations.smoyapt > thresh_sbati) 
                        & (mutations.valeurfonc>thresh_valeur)
                        & ~(mutations.valeurfonc.isna())
                        & (mutations.valeurfonc<1000000)].copy()

final_shape = mutations.shape
print(f"Shape after adjustments and filtering: {final_shape}, that is {final_shape[0]/original_shape[0]: .2%} of original observations\n")

Shape before adjustments and filtering: (1271568, 72)
Shape after adjustments and filtering: (633931, 75), that is  49.85% of original observations



In [52]:
##New process
mutations['latitude'] = mutations.centroid.x
mutations['longitude'] = mutations.centroid.y

# Predictions

In [239]:
for col in ['month', 'year', 'day']:#, 'coddep', 'l_codinsee']:
    mutations[col] = mutations[col].astype('int')

In [278]:
features_1 = [
       'coddep',  'vefa',
       'nblot', 'l_codinsee', 'nbsection', 'nbpar',
       'nbparmut', 'nbsuf', 'sterr', 'nbvolmut',
       'nblocmut', 'nblocmai', 'nblocapt', 'nblocdep',
       'nblocact', 'nbapt1pp', 'nbapt2pp', 'nbapt3pp', 'nbapt4pp', 'nbapt5pp',
       'nbmai1pp', 'nbmai2pp', 'nbmai3pp', 'nbmai4pp', 'nbmai5pp', 'sbati',
       'sbatmai', 'sbatapt', 'sbatact', 'sapt1pp', 'sapt2pp', 'sapt3pp',
       'sapt4pp', 'sapt5pp', 'smai1pp', 'smai2pp', 'smai3pp', 'smai4pp',
       'smai5pp', 'codtypbien', 'libtypbien',
       'month', 'year', 'day', 'nivcentr',
       'population', 'salary', 'smoyapt', 'latitude',
       'longitude'
]

features_2 = [
       'month', 'anneemut', 'day', 'salary', 'population',
       'coddep', 'latitude', 'longitude',
        'l_codinsee', 'sbati', 'sbatmai',
       'sbatapt', 'nivcentr', 
        'sterr', 'smoyapt',
        'nblocmai', 'nblocapt', 'nblocact',
        'nblocmut', 'sbatact'
]

In [279]:
def model_fitting(X_train, y_train, features, scaler_cols, model, scalery):
    
    scalery.fit(y_train)
    y_train = scalery.transform(y_train)

    x_train_scaled = scaler_cols.fit_transform(X_train)
    model.fit(x_train_scaled, y_train.ravel())

    return scaler_cols, model, scalery
    

def test_output(test_set, features, scaler_cols, model, scalery):
    """
    predicting on the test set
    """
    pred = model.predict(scaler_cols.transform(test_set[features]))
    inv_pred = scalery.inverse_transform([pred]).ravel()
    inv_pred = [elt  if elt > 0 else 5000 for elt in inv_pred]

    return inv_pred

def val_prediction(val_set, y_test, features, scaler_cols, model, scalery):
    """
    predicting on the test set
    """
    pred = model.predict(scaler_cols.transform(val_set[features]))
    inv_pred = scalery.inverse_transform([pred]).ravel()
    inv_pred = [elt  if elt > 0 else 5000 for elt in inv_pred]

    print(f"R²: {r2_score(y_test.valeurfonc, inv_pred)}")
    print(f"RMSE: {np.sqrt(mean_squared_error(y_test.valeurfonc, inv_pred))}")
    print(f"MSE: {mean_squared_error(y_test.valeurfonc, inv_pred)}")
    print(f"MAE: {mean_absolute_error(y_test.valeurfonc, inv_pred)}")

    return pred

In [288]:
sample_df = mutations[features_2+['valeurfonc']].copy()#sample(n=1500, random_state=42)
sample_df = sample_df[sample_df.isna().sum(axis=1) == 0].copy()
X = sample_df[features_2].copy()

Y = sample_df[['valeurfonc']].copy()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

scalery = MinMaxScaler()
ct = make_column_transformer(
      (StandardScaler(),
        make_column_selector(dtype_include=np.number)),  # rating
       (OneHotEncoder(),
        make_column_selector(dtype_include=object)))
# model = xgboost.XGBRegressor()
model = RandomForestRegressor()

scaler_cols, model, scalery = model_fitting(X_train, y_train, features_2, ct, model, scalery)

In [289]:
#################
#### VALIDATION SET
#################
val_pred = val_prediction(X_test, y_test, features_2, scaler_cols, model, scalery)

R²: 0.804494662612627
RMSE: 77899.46920433914
MSE: 6068327302.317781
MAE: 45360.145440356086


In [290]:
#################
#### TEST OUTPUT
#################
test_set = pd.read_csv('../../data_mutation/test_set.csv', sep=';')
test_set = mutation_test_process(test_set)
test_set  = niveau_center_connexion(test_set)
test_set = pop_commune_year(test_set)
test_set = salary_connexion(test_set)

test_set['smoymai'] = test_set.sbatmai/test_set.nblocmai
test_set['smoyapt'] = test_set.sbatapt/test_set.nblocapt
test_set['smoyact'] = test_set.sbatact/test_set.nblocact
test_pred = test_output(test_set, features_2, scaler_cols, model, scalery)

hi!


/Users/muhieddineugo/opt/miniconda3/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [251]:
#################
#### PERMUTATION IMPORTANCE
#################
import time
def permutation_importancing(model, result, x_test, y_test, features, barplot = False, boxplot = True):
    """
    Function to compute and plot the permutation
    If you already calculated it, and just want to plot : set result = the_already calculated_result
    else : set result = None
    """
    if result == None:
        start_time = time.time()
        result = permutation_importance(
            model, x_test, y_test, n_repeats=10, random_state=42, n_jobs=2, scoring='roc_auc'
        )
        elapsed_time = time.time() - start_time
        print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")
        forest_importances = pd.Series(result.importances_mean, index=x_test.columns)
        compteur_importancing += 1
    if barplot == True:
        permute_importance = pd.DataFrame(result.values(), index=result.keys(), columns=x_test.columns).T.sort_values('importances_mean',ascending=False)
        fig, ax = plt.subplots(figsize=(10,5))
        permute_importance.importances_mean.plot.bar(yerr=permute_importance.importances_std, ax=ax)
        ax.set_title("Feature importances using permutation on full model")
        ax.set_ylabel("Mean accuracy decrease")

    if boxplot == True:
        #https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html#sphx-glr-auto-examples-inspection-plot-permutation-importance-py
        sorted_importances_idx = result.importances_mean.argsort()

        importances = pd.DataFrame(
            result.importances[sorted_importances_idx].T,
            columns=x_test.columns[sorted_importances_idx],
        )

        fig, ax = plt.subplots(figsize=(11,5))
        importances.plot.box(vert=False, whis=10, ax=ax)
        #ax.set_title("Permutation Importances")
        ax.axvline(x=0, color="k", linestyle="--")
        ax.set_xlabel("Decrease in AUC-ROC score")
        ax.tick_params(axis='y', labelsize=15)
        ax.figure.tight_layout()
        #print('we see that at most the features influence the accuracy by only 0.035 points. Might be corrolated features ! see source ')
    
    return result

In [253]:
# permutation_importancing(model, None, X_test, y_test, features_2, barplot = False, boxplot = True)